In [245]:
# install dependencies: 
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
import torchvision.models as models
# opencv is pre-installed on colab

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import os
import json
import random
from shutil import copyfile
from google.colab import drive
drive.mount('/content/drive')

1.6.0+cu101 False
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [246]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

Image segmentation dataset

In [247]:
path_trainValTest = "/content/drive/My Drive/aim/vision-project/segmentation_dataset/confidence>0.75/trainValTest/"
# Load dataset
with open(path_trainValTest + 'ecommerce-segmentation-dataset-filename.json') as json_file:
    dataset = json.load(json_file)

In [248]:
path_trainValTest = "/content/drive/My Drive/aim/vision-project/segmentation_dataset/confidence>0.75/trainValTest/"

# Load dataset
with open(path_trainValTest + 'ecommerce-segmentation-dataset-filename.json') as json_file:
    dataset = json.load(json_file)

def load_image_from_filename(dataset):
  images = []
  masks = []
  for i in range(len(dataset)):
    if i%25==0:
      print(str(i) + "/" + str(len(dataset)))
    raw_image = tf.keras.preprocessing.image.load_img(dataset[i]["image"])
    arr_image = keras.preprocessing.image.img_to_array(raw_image)
    images.append(arr_image)
    #dataset[i]["image"] = tf.convert_to_tensor(arr_image)

    #raw_mask = tf.keras.preprocessing.image.load_img(dataset[i]["segmentation_mask"])
    #arr_mask = keras.preprocessing.image.img_to_array(raw_mask)
    csv_arr_mask = np.loadtxt(dataset[i]["segmentation_mask"], delimiter=',')
    arr_mask = np.zeros((csv_arr_mask.shape[0],csv_arr_mask.shape[1],1))
    arr_mask[:,:,0] = csv_arr_mask
    masks.append(arr_mask)
    #dataset[i]["segmentation_mask"] = tf.convert_to_tensor(arr_image)

  return images, masks

def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  #input_mask -= 1
  return input_image, input_mask

def load_image_test(img, mask):
  # Image
  input_image = tf.image.resize(img, (128, 128))
  # Mask
  input_mask = tf.image.resize(mask, (128, 128))
  input_image, input_mask = normalize(input_image, input_mask)
  return input_image, input_mask

#train_images, train_masks = load_image_from_filename(dataset["train"])
#val_images, val_masks = load_image_from_filename(dataset["val"])
#test_images, test_masks = load_image_from_filename(dataset["test"])

#test = tf.data.Dataset.from_tensor_slices((test_images, test_masks))
#test = test.map(load_image_test)

In [249]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

LABEL_NAMES = np.asarray([
    'background', 'couch', 'chair', 'bed', 'vase', 'bowl', 'cup',
    'wine-glass', 'potted-plant'
])

In [250]:
len(dataset["test"])

95

Get results with pre-trained models

In [251]:
def preprocess_image(input_image):
  input_image = tf.keras.preprocessing.image.load_img(input_image)
  input_image = keras.preprocessing.image.img_to_array(input_image)
  #input_image = tf.image.resize(arr_image, (128, 128))
  #input_image = input_image/255
  input_image = tf.cast(input_image, tf.float32) / 255.0
  return input_image

def preprocess_mask(input_mask):
  #input_mask = tf.image.resize(input_mask, (128, 128))
  return input_mask

def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    #raw_image = tf.keras.preprocessing.image.load_img(display_list[i])
    #arr_image = keras.preprocessing.image.img_to_array(raw_image)
    #plt.imshow(tf.keras.preprocessing.image.array_to_img(arr_image))
    #plt.imshow(raw_image)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [258]:
model_input_images = []
expected_masks = []

for test_image_id in range(10): #range(len(dataset["test"][0])):
    # Load and preprocess image
    input_image = preprocess_image(dataset["test"][test_image_id]["image"])
    # Channel first
    input_image = torch.tensor(input_image.numpy()).permute(2,0,1)
    model_input_images.append(input_image)

    # Load and resize expcted mask
    expected_mask = preprocess_mask(test_masks[test_image_id])
    expected_masks.append(expected_mask)

In [255]:
# Faster Mask R-CNN model with a ResNet-50-FPN backbone.
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

# To get model evaulation time
def get_model_evaluation_time(model_input_images):
  """
  for input_image in model_input_images:
    # Get model prediction
    predictions = model([input_image])
  """
  predictions = model(model_input_images)

In [256]:
%time get_model_evaluation_time(model_input_images)
# %prun get_model_evaluation_time(model_input_images)

CPU times: user 1min 2s, sys: 6.96 s, total: 1min 9s
Wall time: 1min 9s


In [259]:
# Mean IoU
m = tf.keras.metrics.MeanIoU(num_classes=len(LABEL_NAMES))
m.reset_states()

# To get model evaulation time
for test_image_id in range(len(model_input_images)):

  if test_image_id % 10 == 0:
    print(test_image_id)

  """
  # Load and preprocess image
  input_image = preprocess_image(dataset["test"][test_image_id]["image"])
  # Channel first
  input_image = torch.tensor(input_image.numpy()).permute(2,0,1)

  # Load and resize expcted mask
  expected_mask = preprocess_mask(test_masks[test_image_id])
  """

  input_image = model_input_images[test_image_id]
  expected_mask = expected_masks[test_image_id]

  # Get model prediction
  predictions = model([input_image])
  predicted_mask = predictions[0]["masks"][0].permute(1,2,0).detach().numpy()

  # Threshold soft mask to get 0, 1 labels
  hard_mask = (predicted_mask>0.5).astype(int)
  # Convert binary label to corresponding label category
  predicted_category = COCO_INSTANCE_CATEGORY_NAMES[predictions[0]["labels"][0]]
  label_id = np.where(LABEL_NAMES==predicted_category)[0][0]
  hard_mask[hard_mask==1] = label_id

  # Plot image, mask and predicted mask
  #sample_image, sample_mask = load_image_test(test_images[test_image_id], test_masks[test_image_id])
  #display([sample_image, expected_mask, hard_mask])

  # Compute mean IoU
  m.update_state(hard_mask, expected_mask)

m.result().numpy()

0


0.911264

Etc

In [171]:
hard_mask = (predicted_mask>0.5).astype(int)

predicted_category = COCO_INSTANCE_CATEGORY_NAMES[predictions[0]["labels"][0]]
label_id = np.where(LABEL_NAMES==predicted_category)[0][0]
hard_mask[hard_mask==1] = label_id

t = hard_mask.tolist() #.unique()
flat_list = [item for sublist in t for item in sublist]
flat_list = [item for sublist in flat_list for item in sublist]
np.unique(flat_list)

array([0, 2])